In [ ]:
from datetime import datetime
import os

import numpy as np

from metakkl import utils
from metakkl.train import TrainingDirection

from scripts.utils import defaults, paths
import duffing_train
import duffing_meta_train
import duffing_meta_adapt
import duffing_sim_error
import duffing_sim_error_profile
import duffing_sim_param_error
import duffing_sim_state

dir_path_models = os.path.join(paths.DIR_PATH_MODELS)

# Initial System State Variation

In [ ]:
# Training parameters
num_epochs = 15
batch_size = 32
lr = 1e-3

# Meta learning parameters
meta_batch_size = 16
meta_lr = 3e-5
num_meta_iter = 1000
num_task_iter = 20

# Dataset parameters
param_system_init_state = utils.sample_lhs(limits=np.array([[-1.0, 1.0], [-1.0, 1.0]]), num=50)

param_x_init_error_profile_d_in = np.linspace(-1, 1, 50)
param_x_init_error_profile_d_out = np.linspace(-2, 2, 50)
param_x_init_error_profile_in = (
    np.stack(np.meshgrid(param_x_init_error_profile_d_in, param_x_init_error_profile_d_in), axis=2).reshape((-1, 2)))

defaults.set_validation_seeds()
param_system_init_state_val_in = np.random.uniform(-1.0, 1.0, (50, 2))
param_system_init_state_val_out = np.concatenate((
    np.stack((np.random.uniform(1.0, 2.0, 30), np.random.uniform(-2.0, 2.0, 30)), axis=1),
    np.stack((np.random.uniform(-2.0, -1.0, 30), np.random.uniform(-2.0, 2.0, 30)), axis=1),
    np.stack((np.random.uniform(-1.0, 1.0, 10), np.random.uniform(1.0, 2.0, 10)), axis=1),
    np.stack((np.random.uniform(-1.0, 1.0, 10), np.random.uniform(-2.0, -1.0, 10)), axis=1),
))

system_noise_val = (0.0, 0.1)
system_output_noise_val = (0.0, 0.1)

# Model file paths
model_path_mixed_sequential_x_init = os.path.join(dir_path_models, 'mixed_sequential_x_init.pt')
model_path_mixed_sequential_pde_x_init = os.path.join(dir_path_models, 'mixed_sequential_pde_x_init.pt')
model_path_mixed_parallel_x_init = os.path.join(dir_path_models, 'mixed_parallel_x_init.pt')
model_path_meta_x_init = os.path.join(dir_path_models, 'meta_x_init.pt')
param_model_path_meta_adapted_x_init = os.path.join(dir_path_models, 'meta_adapted_x0={x0}_x1={x1}_x_init.pt')
param_model_path_meta_adapted_x_init_ext_sampling = (
    os.path.join(dir_path_models, 'meta_adapted_x0={x0}_x1={x1}_x_init_ext_sampling.pt'))
param_model_path_meta_adapted_x_init_pre_sampling = (
    os.path.join(dir_path_models, 'meta_adapted_x0={x0}_x1={x1}_x_init_pre_sampling.pt'))
param_model_path_meta_adapted_x_init_ext_pre_sampling = (
    os.path.join(dir_path_models, 'meta_adapted_x0={x0}_x1={x1}_x_init_ext_pre_sampling.pt'))
param_model_path_meta_adapted_x_init_noise = (
    os.path.join(dir_path_models, 'meta_adapted_x0={x0}_x1={x1}_x_init_noise.pt'))

# Log paths
log_path_mixed_sequential_x_init = os.path.join(paths.DIR_PATH_LOGS, 'mixed_sequential_x_init_{time}')
log_path_mixed_sequential_pde_x_init = (
    os.path.join(paths.DIR_PATH_LOGS, 'mixed_sequential_x_init_pde_{time}'))
log_path_mixed_parallel_x_init = os.path.join(paths.DIR_PATH_LOGS, 'mixed_parallel_x_init_{time}')
log_path_meta_x_init = os.path.join(paths.DIR_PATH_LOGS, 'meta_x_init_{time}')

## Training

In [ ]:
# Sequential mixed-task learning

defaults.set_training_seeds()

duffing_train.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state,
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.BIDIRECTIONAL,
    merged=True,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    pde_loss=False,
    model_path=None,
    save_path=model_path_mixed_sequential_x_init,
    log_dir_path=log_path_mixed_sequential_x_init.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

In [ ]:
# Sequential mixed-task learning with PDE constraint

defaults.set_training_seeds()

duffing_train.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state,
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.BIDIRECTIONAL,
    merged=True,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    pde_loss=True,
    model_path=None,
    save_path=model_path_mixed_sequential_pde_x_init,
    log_dir_path=log_path_mixed_sequential_pde_x_init.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

In [ ]:
# Parallel mixed-task learning

defaults.set_training_seeds()

duffing_train.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state,
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.BIDIRECTIONAL,
    merged=False,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    pde_loss=False,
    model_path=None,
    save_path=model_path_mixed_parallel_x_init,
    log_dir_path=log_path_mixed_parallel_x_init.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

In [ ]:
# Meta-learning for system output adaptation

defaults.set_training_seeds()

duffing_meta_train.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state,
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    task_output_loss=True,
    meta_batch_size=meta_batch_size,
    meta_lr=meta_lr,
    num_meta_iter=num_meta_iter,
    task_batch_size=batch_size,
    random_batch_offset=True,
    shuffle_batch=False,
    task_lr=None,
    num_task_iter=num_task_iter,
    model_path=model_path_mixed_parallel_x_init,
    save_path=model_path_meta_x_init,
    log_dir_path=log_path_meta_x_init.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

## Error averaged over multiple initial system states

In [ ]:
# System output adaptation
# Initial system state inside of training region
# Minimum sampling period

defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state_val_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=False,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init
)

In [ ]:
# Simulate error
# Initial system state inside of training region

defaults.set_validation_seeds()

duffing_sim_error.simulate(
    lmbda=defaults.DUFFING_SYSTEM_LMBDA,
    param_system_init_state=param_system_init_state_val_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    model_paths=(
        model_path_mixed_sequential_x_init,
        model_path_mixed_sequential_pde_x_init,
        model_path_mixed_parallel_x_init,
    ),
    param_model_paths=(
        param_model_path_meta_adapted_x_init,
    ),
    data_path=paths.DATA_PATH_ERROR_X_INIT_IN
)

In [ ]:
# System output adaptation
# Initial system state outside of training region
# Minimum sampling period

defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state_val_out,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=False,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init
)

In [ ]:
# Simulate error
# Initial system state outside of training region

defaults.set_validation_seeds()

duffing_sim_error.simulate(
    lmbda=defaults.DUFFING_SYSTEM_LMBDA,
    param_system_init_state=param_system_init_state_val_out,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    model_paths=(
        model_path_mixed_sequential_x_init,
        model_path_mixed_sequential_pde_x_init,
        model_path_mixed_parallel_x_init,
    ),
    param_model_paths=(
        param_model_path_meta_adapted_x_init,
    ),
    data_path=paths.DATA_PATH_ERROR_X_INIT_OUT
)

## Error profile over initial system state

In [ ]:
defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_x_init_error_profile_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=True,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init
)

In [ ]:
defaults.set_validation_seeds()

duffing_sim_error_profile.simulate(
    lmbda=defaults.DUFFING_SYSTEM_LMBDA,
    param_system_init_state_0=param_x_init_error_profile_d_in,
    param_system_init_state_1=param_x_init_error_profile_d_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    param_model_path=param_model_path_meta_adapted_x_init,
    data_path=paths.DATA_PATH_ERROR_PROFILE_META_ADAPTED_X_INIT_IN
)

## Error averaged over multiple initial system states with different sampling periods

In [ ]:
# System output adaptation
# Initial system state inside of training region
# Extended sampling period

defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state_val_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=True,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init_ext_sampling
)

In [ ]:
# System output adaptation
# Initial system state inside of training region
# Delayed sampling period

defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state_val_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=False,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init_pre_sampling
)

In [ ]:
# System output adaptation
# Initial system state inside of training region
# Delayed and extended sampling period

defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=param_system_init_state_val_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=True,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init_ext_pre_sampling
)

In [ ]:
defaults.set_validation_seeds()

duffing_sim_error.simulate(
    lmbda=defaults.DUFFING_SYSTEM_LMBDA,
    param_system_init_state=param_system_init_state_val_in,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    model_paths=(),
    param_model_paths=(
        param_model_path_meta_adapted_x_init,
        param_model_path_meta_adapted_x_init_ext_sampling,
        param_model_path_meta_adapted_x_init_pre_sampling,
        param_model_path_meta_adapted_x_init_ext_pre_sampling,
    ),
    data_path=paths.DATA_PATH_ERROR_META_ADAPTED_X_INIT_SAMPLING
)

## Error for specific initial system state

In [ ]:
defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=(defaults.DUFFING_SYSTEM_INIT_STATE,),
    system_noise=None,
    system_output_noise=None,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=True,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init
)

defaults.set_validation_seeds()

duffing_sim_state.simulate(
    lmbda=defaults.DUFFING_SYSTEM_LMBDA,
    system_init_state=defaults.DUFFING_SYSTEM_INIT_STATE,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    model_paths=(
        param_model_path_meta_adapted_x_init.format(x0=defaults.DUFFING_SYSTEM_INIT_STATE[0],
                                                    x1=defaults.DUFFING_SYSTEM_INIT_STATE[1]),
    ),
    data_path=paths.DATA_PATH_STATE_META_ADAPTED_X_INIT
)

In [ ]:
defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=(defaults.DUFFING_SYSTEM_LMBDA,),
    param_system_init_state=(defaults.DUFFING_SYSTEM_INIT_STATE,),
    system_noise=system_noise_val,
    system_output_noise=system_output_noise_val,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=batch_size,
    shuffle=True,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_x_init,
    param_save_path=param_model_path_meta_adapted_x_init_noise
)

defaults.set_validation_seeds()

duffing_sim_state.simulate(
    lmbda=defaults.DUFFING_SYSTEM_LMBDA,
    system_init_state=defaults.DUFFING_SYSTEM_INIT_STATE,
    system_noise=system_noise_val,
    system_output_noise=system_output_noise_val,
    observer_init_state=None,
    model_paths=(
        param_model_path_meta_adapted_x_init.format(x0=defaults.DUFFING_SYSTEM_INIT_STATE[0],
                                                    x1=defaults.DUFFING_SYSTEM_INIT_STATE[1]),
    ),
    data_path=paths.DATA_PATH_STATE_META_ADAPTED_X_INIT_NOISE
)

# System Parameter Variation

In [ ]:
# Training parameters
num_epochs = 20
batch_size = 32
lr = 1e-3

# Meta learning parameters
meta_batch_size = 2
meta_task_batch_size = 4
meta_lr = 3e-4
num_meta_iter = 1000
num_task_iter = 100

# Dataset parameters
param_lmbda = np.linspace(1.0, 5.0, 5)

defaults.set_validation_seeds()
param_lmbda_val = np.sort(np.random.uniform(1.0, 5.0, 200))

system_noise_val = (0.0, 0.1)
system_output_noise_val = (0.0, 0.1)

# Model file paths
model_path_mixed_sequential_lmbda = os.path.join(dir_path_models, 'mixed_sequential_lmbda.pt')
model_path_mixed_sequential_pde_lmbda = os.path.join(dir_path_models, 'mixed_sequential_pde_lmbda.pt')
model_path_mixed_parallel_lmbda = os.path.join(dir_path_models, 'mixed_parallel_lmbda.pt')
model_path_mixed_parallel_forward_lmbda = os.path.join(dir_path_models, 'mixed_parallel_forward_lmbda.pt')
model_path_meta_lmbda = os.path.join(dir_path_models, 'meta_lmbda.pt')
param_model_path_meta_adapted_lmbda = os.path.join(dir_path_models, 'meta_adapted_lmbda={lmbda}.pt')
param_model_path_meta_adapted_lmbda_sequential = (
    os.path.join(dir_path_models, 'meta_adapted_lmbda={lmbda}_sequential.pt'))
param_model_path_meta_adapted_lmbda_pre = os.path.join(dir_path_models, 'meta_adapted_lmbda={lmbda}_pre.pt')
param_model_path_meta_adapted_lmbda_pre_sequential = (
    os.path.join(dir_path_models, 'meta_adapted_lmbda={lmbda}_pre_sequential.pt'))

# Log paths
log_path_mixed_sequential_lmbda = os.path.join(paths.DIR_PATH_LOGS, 'mixed_sequential_lmbda_{time}')
log_path_mixed_sequential_pde_lmbda = (
    os.path.join(paths.DIR_PATH_LOGS, 'mixed_sequential_lmbda_pde_{time}'))
log_path_mixed_parallel_lmbda = os.path.join(paths.DIR_PATH_LOGS, 'mixed_parallel_lmbda_{time}')
log_path_meta_lmbda = os.path.join(paths.DIR_PATH_LOGS, 'meta_lmbda_{time}')

In [ ]:
# Sequential mixed-task learning

defaults.set_training_seeds()

duffing_train.run(
    param_lmbda=param_lmbda,
    param_system_init_state=(
        defaults.DUFFING_SYSTEM_INIT_STATE,
    ),
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.BIDIRECTIONAL,
    merged=True,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    pde_loss=False,
    model_path=None,
    save_path=model_path_mixed_sequential_lmbda,
    log_dir_path=log_path_mixed_sequential_lmbda.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

In [ ]:
# Sequential mixed-task learning with PDE constraint

defaults.set_training_seeds()

duffing_train.run(
    param_lmbda=param_lmbda,
    param_system_init_state=(defaults.DUFFING_SYSTEM_INIT_STATE,),
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.BIDIRECTIONAL,
    merged=True,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    pde_loss=True,
    model_path=None,
    save_path=model_path_mixed_sequential_pde_lmbda,
    log_dir_path=log_path_mixed_sequential_pde_lmbda.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

In [ ]:
# Parallel mixed-task learning

defaults.set_training_seeds()

duffing_train.run(
    param_lmbda=param_lmbda,
    param_system_init_state=(defaults.DUFFING_SYSTEM_INIT_STATE,),
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.BIDIRECTIONAL,
    merged=False,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    pde_loss=False,
    model_path=None,
    save_path=model_path_mixed_parallel_lmbda,
    log_dir_path=log_path_mixed_parallel_lmbda.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

In [ ]:
# Meta-learning for system output adaptation

defaults.set_training_seeds()

duffing_meta_train.run(
    param_lmbda=param_lmbda,
    param_system_init_state=(
        defaults.DUFFING_SYSTEM_INIT_STATE,
    ),
    system_noise=None,
    system_output_noise=None,
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    task_output_loss=True,
    meta_batch_size=meta_batch_size,
    meta_lr=meta_lr,
    num_meta_iter=num_meta_iter,
    task_batch_size=meta_task_batch_size,
    random_batch_offset=True,
    shuffle_batch=False,
    task_lr=None,
    num_task_iter=num_task_iter,
    model_path=model_path_mixed_parallel_lmbda,
    save_path=model_path_meta_lmbda,
    log_dir_path=log_path_meta_lmbda.format(time=datetime.now().strftime("%b%d_%H-%M-%S"))
)

## Error over parameter range

In [ ]:
# System output adaptation

defaults.set_validation_seeds()

duffing_meta_adapt.run(
    param_lmbda=param_lmbda_val,
    param_system_init_state=(
        defaults.DUFFING_SYSTEM_INIT_STATE,
    ),
    system_noise=None,
    system_output_noise=None,
    observer_init_state=np.empty(0),
    direction=TrainingDirection.UNIDIRECTIONAL_INVERSE,
    merged=False,
    output_loss=True,
    batch_size=meta_task_batch_size,
    shuffle=False,
    lr=None,
    num_optim_steps=num_task_iter,
    model_path=model_path_meta_lmbda,
    param_save_path=param_model_path_meta_adapted_lmbda
)

In [ ]:
defaults.set_validation_seeds()

duffing_sim_param_error.simulate(
    param_lmbda=param_lmbda_val,
    system_init_state=defaults.DUFFING_SYSTEM_INIT_STATE,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    model_paths=(
        model_path_mixed_sequential_lmbda,
        model_path_mixed_sequential_pde_lmbda,
        model_path_mixed_parallel_lmbda,
    ),
    param_model_paths=(
        param_model_path_meta_adapted_lmbda,
    ),
    sim_error_x=True,
    sim_error_z=True,
    data_path=paths.DATA_PATH_PARAM_ERROR_LMBDA
)

In [ ]:
defaults.set_validation_seeds()

duffing_sim_param_error.simulate(
    param_lmbda=param_lmbda_val,
    system_init_state=defaults.DUFFING_SYSTEM_INIT_STATE,
    system_noise=None,
    system_output_noise=None,
    observer_init_state=None,
    model_paths=(
        model_path_mixed_parallel_lmbda,
    ),
    param_model_paths=(
        param_model_path_meta_adapted_lmbda,
    ),
    sim_error_x=True,
    sim_error_z=True,
    data_path=paths.DATA_PATH_PARAM_ERROR_LMBDA_PARALLEL
)